# 人脸识别
这个练习的目的是利用主成分分析（PCA）和线性判别分析（LDA）进行图像的人脸识别。在编码前先阅读所有指令。

## 1. 导入数据  
下载face.zip并从face.zip提取图像。你会发现一些人脸图像来自CMU PIE数据库。这里有10个人，每个人在24种不同的照明条件下拍摄，共有240幅图像。为了方便起见，这些图像在训练和测试目录中被分成两个相等的集合。  
图像的文件名ppppp_xx_yy.bmp，这里ppppp表示人的身份；xx表示头部方向（在这个练习中,所有的方向都是正面的（xx = 27））; YY表示照明条件。所有图像都被裁剪并对齐，它们的高度、宽度分别为160和140像素。

## 2. 特征提取  
特征提取在人脸识别中起着重要作用。好的特性应该能够区分不同的用户。  
在这部分中，您将学习如何提取PCA特征和LDA特征。
### 2.1 PCA特征
这部分将指导您从图像的训练集学习PCA特征提取器。您还将学习如何用图像提取PCA特征。  
遵照指示：  
1. 读取数据。通过将参数设置为训练文件夹的路径，使用read_faces读取所有训练图像。现在你应该有一个$22400\times120$的脸矩阵，它的列是训练目录中所有的人脸图像，以及相应的标签范围从0到9。  
2. 训练PCA。使用mypca函数，计算PCA投影矩阵$W$，全局均值向量$m_e$和特征值向量。学习代码- mypca，理解这是如何实现的。特别是，注意内积技巧的使用避免出现“内存不足”错误。    
3. 选择的特征维度。通过公式$ W_e = W[:,:K]$只保留前$K(K=30)$特征脸。    
4. 投影图像。对于一个人脸图像x，通过公式$y = W_e^T(x-m_e)$将它投影到PCA空间中。y是x的PCA特征表示。 
5. 反向投影PCA特征。众所周知，PCA也经常被用作降维方法。所以我们可以在已学习的PCA空间由特征向量y重建图像x：$x = W_ey+m_e$。

In [ ]:
import os
import numpy as np
import scipy.linalg as linalg
import cv2
import operator
def ComputeNorm(x):
    # function r=ComputeNorm(x)
    # computes vector norms of x
    # x: d x m matrix, each column a vector
    # r: 1 x m matrix, each the corresponding norm (L2)

    [row, col] = x.shape
    r = np.zeros((1,col))

    for i in range(col):
        r[0,i] = linalg.norm(x[:,i])
    return r

def myPCA(A):
    # function [W,LL,m]=mypca(A)
    # computes PCA of matrix A
    # A: D by N data matrix. Each column is a random vector
    # W: D by K matrix whose columns are the principal components in decreasing order
    # LL: eigenvalues
    # m: mean of columns of A

    # Note: "lambda" is a Python reserved word


    # compute mean, and subtract mean from every column
    [r,c] = A.shape
    m = np.mean(A,1)
    A = A - np.tile(m, (c,1)).T
    B = np.dot(A.T, A)
    [d,v] = linalg.eig(B)

    # sort d in descending order
    order_index = np.argsort(d)
    order_index =  order_index[::-1]
    d = d[order_index]
    v = v[:, order_index]

    # compute eigenvectors of scatter matrix
    W = np.dot(A,v)
    Wnorm = ComputeNorm(W)

    W1 = np.tile(Wnorm, (r, 1))
    W2 = W / W1
    
    LL = d[0:-1]

    W = W2[:,0:-1]      #omit last column, which is the nullspace
    
    return W, LL, m

def read_faces(directory):
    # function faces = read_faces(directory)
    # Browse the directory, read image files and store faces in a matrix
    # faces: face matrix in which each colummn is a colummn vector for 1 face image
    # idLabels: corresponding ids for face matrix

    A = []  # A will store list of image vectors
    Label = [] # Label will store list of identity label
 
    # browsing the directory
    for f in os.listdir(directory):
        if not f[-3:] =='bmp':
            continue
        infile = os.path.join(directory, f)
        im = cv2.imread(infile, 0)
        # turn an array into vector
        im_vec = np.reshape(im, -1)
        A.append(im_vec)
        name = f.split('_')[0][-1]
        Label.append(int(name))

    faces = np.array(A, dtype=np.float32)
    faces = faces.T
    idLabel = np.array(Label)

    return faces,idLabel

def float2uint8(arr):
    mmin = arr.min()
    mmax = arr.max()
    arr = (arr-mmin)/(mmax-mmin)*255
    arr = np.uint8(arr)
    return arr

### 2.2 LDA特征
这部分将指导您从一组图像训练中学习LDA特征提取器。您还将学习如何用提取器提取图像的LDA特征。
由于计算代价，在LDA特征提取前经常采用PCA来对输入降维。根据PCA投影的结果，按照指示进行操作：  
1. 降维。将输入(22400维)降到$K_1$ ($K_1=90$)维。通过$W_1 = W[:,:K_1]$ 只保留前$K_1$个特征脸，W是在PCA特征提取的结果。降低x的维度通过$x'=W_1^T(x-m_e)$。对所有的人脸图像进行降维。现在应该有一个$K_1\times 120$维的矩阵X。    
2. 训练LDA。在矩阵X和对应的labels上训练LDA，计算得到LDA的投影矩阵$W_f$和在LDA的空间C上的各类中心。     
3. 投影图像。对于一个给定的人脸图像x，通过式$y = W_f^TW_1^T(x-m_e)$投影到LDA空间。

In [ ]:
def myLDA(A,Labels):
    # function [W,m]=myLDA(A,Label)
    # computes LDA of matrix A
    # A: D by N data matrix. Each column is a random vector
    # W: D by K matrix whose columns are the principal components in decreasing order
    # m: mean of each projection
    classLabels = np.unique(Labels)
    classNum = len(classLabels)
    dim,datanum = A.shape
    totalMean = np.mean(A,1)
    partition = [np.where(Labels==label)[0] for label in classLabels]
    classMean = [(np.mean(A[:,idx],1),len(idx)) for idx in partition]

    #compute the within-class scatter matrix
    W = np.zeros((dim,dim))
    for idx in partition:
        W += np.cov(A[:,idx],rowvar=1)*len(idx)

    #compute the between-class scatter matrix
    B = np.zeros((dim,dim))
    for mu,class_size in classMean:
        offset = mu - totalMean
        B += np.outer(offset,offset)*class_size

    #solve the generalized eigenvalue problem for discriminant directions
    ew, ev = linalg.eig(B, W)

    sorted_pairs = sorted(enumerate(ew), key=operator.itemgetter(1), reverse=True)
    selected_ind = [ind for ind,val in sorted_pairs[:classNum-1]]
    LDAW = ev[:,selected_ind]
    Centers = [np.dot(mu,LDAW) for mu,class_size in classMean]
    Centers = np.array(Centers).T
    return LDAW, Centers, classLabels

## 3. 识别系统。
根据我们学习的内容，人脸识别系统具有两个部分：注册(enrollment)和识别(identification)。  
### 3.1 Enrollment Session
建立人脸识别系统的第一部分是注册所有用户。也就是说，我们需要将所有用户的面部信息存储在系统数据库中。为了简单起见，我们可以使用每个用户的平均特征来表示他们的身份。基于第二节中的特征提取方法。按照如下指示进行操作：    
1. 将所有训练图像投影到相应的特征空间中。
2. 对于每个人，计算所有特征向量的平均z，按列存储这些向量在numpy矩阵Z，矩阵Z的第i列对应着身份标签-i，检查你的Z矩阵应该是$D_f\times 10$维的矩阵，其中$D_f$是特征维度。在我们的LDA的任务中，Z是myLDA函数返回的类中心。  

### 3.2 Identification Session  
你的面部识别系统现在已准备就绪。 为了识别新的脸部图像x(re-shaped into a vector)，首先通过将其投影到特征空间中来提取其特征。 然后使用欧几里得距离度量来搜索最接近于y的模板。最近模板的索引揭示了x的身份。 例如，如果最近的模板是第2列，则预测的id是classLabel [1]。    

您现在可以评估识别系统的性能。 使用测试目录中的图像，如上所述,使用您的识别系统识别每个图像。  

混淆矩阵(confusion matrix)是评估灯光识别系统性能的有用方法。 $M\times M$的 混淆矩阵C中的每个元素$c_{ij}$是系统将来自人i的图像标识为人j的次数。 因此完美的识别系统应该产生一个对角线的混淆矩阵。 C中的任何非零对角线元素都表示一个错误。 识别系统的总体准确度可以通过迹线除以所有元素的总和来计算。
计算$10\times10$混淆矩阵。 首先，将混淆矩阵中的所有条目初始化为0.然后，对于每个测试图像，让预测的id为您的分类器输出的身份，并让实际id为测试图像的实际身份。 将1添加到混淆矩阵的实际第id行和预测的第id列中的条目。

In [ ]:
import matplotlib.pyplot as plt
import cv2
########################################
def data_new(k,W,A,m):
    # Function newA=data_new(k,W,A)
    # compute the reducing matrix
    # Wk: the matrix composed of the first k eigenvectors
    # A: is the list of image vectors- mean of column of A
    b1=A.shape[1]
    A=A - np.tile(m, (b1,1)).T
    Wk=W[:,:k]
    newA=np.dot(Wk.T,A)
    return newA
#########################################
def centerPCA(A):
    centrA=np.zeros((10,30))
    b1=A.shape[1]
    AT=A.T
    for i in range(b1):
        ii=i//12
        centrA[ii]=centrA[ii]+AT[i]
    return centrA.T/12
######################################
def confusionmatrix(A,B):
    cm=np.zeros((10,10)) 
    b1=B.shape[1]
    distance=np.zeros(10)
    AT=A.T
    BT=B.T
    for i in range(b1):
        for j in range(10): #test 
            distance[j]=np.linalg.norm(AT[j] - BT[i])  #distance#print(dis)
        order_index = np.argsort(distance) #sort
        index = order_index[0]  # find the index of the min
        cm[ i//12 ][ index ] = cm[ i//12 ][ index ] + 1
    return cm
#################################
def fusiondata(A,B,alpha):
    C=np.vstack((alpha*A,(1-alpha)*B))
    return C

## 4. 
在代码中实现以下任务：  
1. 训练PCA特征提取器并利用训练数据集建立基于PCA的标识符。用混淆矩阵评估测试数据集的标识符。  
2. 基于主成分分析的任务，可视化平均脸$m_e$和前八个特征脸（主成分对应的前八个最大特征值）在$W_e$中如下：使用numpy-reshape功能重建$160\times140$的二维矩阵，然后用pyplot subplot命令创建$3\times3$的子图，在这些图上显示这些特征脸。使用提供的float2uint8功能尺度化像素值0和255之间。在右下角的图，显示的平均脸$m_e$。对每个子图命名“eigenface1”，“eigenface2”，...，“mean”。  
3. 使用训练数据集训练LDA特征提取器和基于LDA的识别器。在测试集中评估识别器，用混淆矩阵表示结果。  
4. 基于LDA的任务，可视化每个类别的平均脸：首先，项目中心向量空间空间：LDA PCA CP = WF CF；其次，项目FROMP PCA特征向量空间到图像空间：CR = WECP + M，其中M是平铺于我遇见的尺寸一致性；最后，将各含中心的形象与情节在2 5次要情节与片名为“center1”…“center10”。  
5. 为了提高识别系统的性能，通常采用融合方案。融合方案可分为原始数据级、特征级和评分级三个层次。在这个任务中，请实现融合方法在特征级。基于PCA和LDA特征向量的特征向量你们YF，融合后的特征向量可以构造

In [ ]:
##########################
'''1'''
#####oringal data###
trdata,trlabel=read_faces('train')
tedata,telabel=read_faces('test')

####PCA data######
Wtr,Ltr,mtr=myPCA(trdata)
tr=data_new(30,Wtr,trdata,mtr)
te=data_new(30,Wtr,tedata,mtr)
#the class-mean of PCA is: Z  ##
Z=centerPCA(tr,trlabel,30)
PCAcm=confusionmatrix(Z,te)
AR_pca=np.trace(PCAcm)/sum(sum(PCAcm))
print('the cofusion matrix of PCA is:\n',PCAcm)
print('the accurate of PCA is:',round(AR_pca,3))
###########eigenvalues-faces################
'''2'''
#mean-face
fig=plt.figure()
plt.gray()
plt.subplot(3,3,9)
mtr1=float2uint8(mtr)
plt.imshow(mtr1.reshape((160,140)))
plt.title('mean')
#eigen-face 1-8
for i in range(0,8):
    plt.subplot(3,3,i+1)
    Wtr1=float2uint8(Wtr[:,i])
    plt.imshow(Wtr1.reshape((160,140))) 
    plt.title('Eigenface'+str(i+1))
fig.subplots_adjust(hspace=0.6)
plt.savefig("Eigenfaces.png")
plt.show()
############LDA#######
'''3'''
### LDA data####
LDtr=data_new(90,Wtr,trdata,mtr)
LDte=data_new(90,Wtr,tedata,mtr)
LDAW, Centers, classLabels=myLDA(LDtr,trlabel)
LDAtr=np.dot(LDAW.T,LDtr)
LDAte=np.dot(LDAW.T,LDte)
#the class-mean of LDA is: Centers###
LDAcm=confusionmatrix(Centers,LDAte)
AR_lda=np.trace(LDAcm)/sum(sum(LDAcm))
print('the cofusion matrix of LDA is:\n',LDAcm)
print('the accurate of LDA is:',round(AR_lda,3))
##############LDA plot##
'''4'''
Cp=np.dot(LDAW,Centers)
tiletime=Cp.shape[1]
Cr=np.dot(Wtr[:,:90],Cp)+np.tile(mtr, (tiletime,1)).T
##plot center##
fig = plt.figure()
plt.gray()
for i in range(0,10):
    plt.subplot(2,5,i+1)
    Wtr1=float2uint8(Cr[:,i])
    plt.imshow(Wtr1.reshape((160,140))) 
    plt.title('Center'+str(i+1))
fig.subplots_adjust(wspace=0.5)
plt.savefig("Centerfaces.png")
plt.show()
###################
'''5'''
'''alpha=0.5'''
fuste=fusiondata(te,LDAte,0.5)
# the fusion-class mean of 0.5PCA+0.5LDA#
fuZ=fusiondata(Z,Centers,0.5)
fusioncm=confusionmatrix(fuZ,fuste)
AR_fus=np.trace(fusioncm)/sum(sum(fusioncm))
print('the cofusion matrix of fusion is:\n',fusioncm)
print('the accurate of fusion is:',round(AR_fus,3))
########################
'''6'''
accuracy=np.zeros((9,1))
alpha=np.zeros((9,1))
for i in range(0,9):
    alpha[i]=0.1*(i+1)
    fuste1=fusiondata(te,LDAte,alpha[i])
    fuZ1=fusiondata(Z,Centers,alpha[i])
    fusioncm1=confusionmatrix(fuZ1,fuste1)
    accuracy[i]=round(np.trace(fusioncm1)/sum(sum(fusioncm1)),3)
print(accuracy, alpha)
fig = plt.figure()
plt.scatter(alpha, accuracy, marker = '+', color = 'c') 
plt.title('accuracy versus alpha')
plt.xlabel('alpha')
plt.ylabel('accuracy')
plt.savefig('accuracy-alpha.png')
plt.show()